# 使用 Pytorch 以 ADE20K 训练集训练 DeepLabV3Plus-RESNET101

## 1. 基础准备

### 1.1 （第一次或在虚拟环境运行）安装相关库


In [3]:
%pip install numpy<2.0
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install opencv-python
%pip install matplotlib
%pip install tqdm

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple




[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ϵͳ�Ҳ���ָ�����ļ���


### 1.2 检查 CUDA 版本


In [7]:
!nvidia-smi

Mon Jul 22 22:19:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.80                 Driver Version: 546.80       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   49C    P8               9W / 140W |     90MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 2. 搭建模型

### 2.1 导入相关库


In [1]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models.segmentation as models
from pathlib import Path
from tqdm import tqdm

### 2.2 定义数据集类

数据集可在 [ADE20K](http://data.csail.mit.edu/places/ADEchallenge/ADEChallengeData2016.zip) 下载


In [2]:
DATASETPATH = Path("dataset/ade20k/ADEChallengeData2016")  # 数据集位置

class ADE20kDataset(Dataset):
    def __init__(self, transforms=None):
        self.transforms = transforms
        self.images = sorted(os.listdir(os.path.join(DATASETPATH, "images", "training")))
        self.masks = sorted(os.listdir(os.path.join(DATASETPATH, "annotations", "training")))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(DATASETPATH, "images", "training", self.images[idx])
        mask_path = os.path.join(DATASETPATH, "annotations", "training", self.masks[idx])

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)

        if self.transforms:
            image = self.transforms(image)
            mask = self.transforms(mask)

        return image, mask

### 2.3 数据预处理和加载


In [7]:
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ADE20kDataset(transforms=data_transforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)

### 2.4 模型预加载

此处模型使用 Pytorch 中的 DeepLabV3Plus 模型


In [5]:
model = models.deeplabv3_resnet101(pretrained=False, num_classes=151) 

# 转移到cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cuda')
model = model.to(device)

c:\Users\Sparr\Desktop\pytorch-deeplabv3plus-resnet101-ade20k\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Sparr\Desktop\pytorch-deeplabv3plus-resnet101-ade20k\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


### 2.5 训练模型


In [8]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.05)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, masks in tqdm(train_loader):
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, masks.long())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print('Training complete')

  0%|          | 0/5053 [00:00<?, ?it/s]